> <p><small><small>Copyright 2021 DeepMind Technologies Limited.</p>
> <p><small><small> Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at </p>
> <p><small><small> <a href="https://www.apache.org/licenses/LICENSE-2.0">https://www.apache.org/licenses/LICENSE-2.0</a> </p>
> <p><small><small> Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License. </p>


# Generative Models


**Aim**

Our goal is to introduce you, students interested in STEM and or Artificial Intelligence, to the fascinating world of generative models; algorithms that are able to generate realistic novel creations, such as texts, music or, as in this colab, images.

**Disclaimer**

This code is intended for educational purposes, and in the name of readability for a non-technical audience does not always follow best practices for software engineering.

**Links to resources**
- [What is Colab?](https://colab.research.google.com) If you have never used Colab before, get started here!

# Can you tell which of these images are real?

Let's play a game. We'll show you some images, some of them have been generated using Artificial Intelligence (AI), some of of them are real.

Will you be able to spot the difference?

First, let's set up the quiz by running the cell below:


In [ ]:
#@title Setting up the "Real or Fake?" quiz.
#@markdown This may take a minute. 
%%capture
# Add all the pip installs of modules necessary to run the colab
%reset -f

!apt-get update
!apt-get install pip

# You can directly pip install packages
# e.g. !pip install dm-acme
# or you can clone the repo from GithHub
# e.g. !git clone https://github.com/deepmind/acme
!pip install ipywidgets


#License for the emoji lib: https://github.com/carpedm20/emoji/blob/master/LICENSE.txt
!pip install emoji --upgrade

# Import all the modules needed to run the colab here 
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


import io
import IPython.display
import math
from matplotlib import pylab
from matplotlib import pyplot as plt
from matplotlib import rc
rc('animation', html='jshtml')
import matplotlib.animation as animation
import numpy as np
import PIL.Image
from scipy.stats import truncnorm
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
from google.colab import widgets
import time
from skimage import io
import ipywidgets
import emoji

# Set up the quiz.

#Create the GAN source of data. Copied from https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/biggan_generation_with_tf_hub.ipynb#scrollTo=ry62-8SWfuds
# BigGAN (original) models
tf.reset_default_graph()
module_path = 'https://tfhub.dev/deepmind/biggan-128/2'  # 128x128 BigGAN
module = hub.Module(module_path)
inputs = {k: tf.placeholder(v.dtype, v.get_shape().as_list(), k)
          for k, v in module.get_input_info_dict().items()}
output = module(inputs)

# Helper functions for BigGAN. Copied from https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/biggan_generation_with_tf_hub.ipynb#scrollTo=ry62-8SWfuds
input_z = inputs['z']
input_y = inputs['y']
input_trunc = inputs['truncation']

dim_z = input_z.shape.as_list()[1]
vocab_size = input_y.shape.as_list()[1]

def truncated_z_sample(batch_size, truncation=1., seed=None):
  """Draws noise samples from a truncated Gaussian distribution."""
  state = None if seed is None else np.random.RandomState(seed)
  values = truncnorm.rvs(-2, 2, size=(batch_size, dim_z), random_state=state)
  return truncation * values

def one_hot(index, vocab_size=vocab_size):
  """Creates one-hot vectors to encode the label (or category).

  e.g. one_hot(2, 5) = [0, 0, 1, 0, 0].
  """
  index = np.asarray(index)
  if len(index.shape) == 0:
    index = np.asarray([index])
  assert len(index.shape) == 1
  num = index.shape[0]
  output = np.zeros((num, vocab_size), dtype=np.float32)
  output[np.arange(num), index] = 1
  return output

def one_hot_if_needed(label, vocab_size=vocab_size):
  """Creates one-hot vectors to encode the label if it is not already one-hot.

  e.g. one_hot(label=2, vocab_size=5) = [0, 0, 1, 0, 0].
  e.g. one_hot(label=[0, 0, 1, 0, 0], vocab_size=5) = [0, 0, 1, 0, 0].
  """
  label = np.asarray(label)
  if len(label.shape) <= 1:
    label = one_hot(label, vocab_size)
  assert len(label.shape) == 2
  return label

def sample(sess, noise, label, truncation=1., batch_size=8,
           vocab_size=vocab_size):
  """Generate an image by sampling the generative model."""
  noise = np.asarray(noise)
  label = np.asarray(label)
  num = noise.shape[0]
  if len(label.shape) == 0:
    label = np.asarray([label] * num)
  label = one_hot_if_needed(label, vocab_size)
  ims = []
  for batch_start in range(0, num, batch_size):
    s = slice(batch_start, min(num, batch_start + batch_size))
    feed_dict = {input_z: noise[s], input_y: label[s], input_trunc: truncation}
    ims.append(sess.run(output, feed_dict=feed_dict))
  ims = np.concatenate(ims, axis=0)
  assert ims.shape[0] == num
  ims = np.clip(((ims + 1) / 2.0) * 256, 0, 255)
  ims = np.uint8(ims)
  return ims

def interpolate(A, B, num_interps):
  """Linearly interpolate between vectors A and B.
  
  e.g. interpolation = a*A + (1 - a)*B for a in [0, ..., 1]
  """
  alphas = np.linspace(0, 1, num_interps)
  return np.array([(1-a)*A + a*B for a in alphas])

def imgrid(imarray, cols=5, pad=1):
  """Function to display images in a grid."""
  pad = int(pad)
  assert pad >= 0
  cols = int(cols)
  assert cols >= 1
  N, H, W, C = imarray.shape
  rows = N // cols + int(N % cols != 0)
  batch_pad = rows * cols - N
  assert batch_pad >= 0
  post_pad = [batch_pad, pad, pad, 0]
  pad_arg = [[0, p] for p in post_pad]
  imarray = np.pad(imarray, pad_arg, 'constant', constant_values=255)
  H += pad
  W += pad
  grid = (imarray
          .reshape(rows, cols, H, W, C)
          .transpose(0, 2, 1, 3, 4)
          .reshape(rows*H, cols*W, C))
  if pad:
    grid = grid[:-pad, :-pad]
  return grid

# URLs are taken from ImageNet.
real_image_urls = ['http://static.flickr.com/27/65922179_fe6774cccf.jpg', 'https://farm3.static.flickr.com/2223/1589046073_5420d1ca1d.jpg', 'http://farm3.static.flickr.com/2327/2047231704_fcc0270193.jpg', 'http://www.fungimag.com/summer-08-articles/shiitakesepia2_sm.jpg', 'http://static.flickr.com/39/78374930_f0260bb501.jpg', 'http://farm2.static.flickr.com/1104/526094174_f627d7d2b9.jpg', 'http://farm1.static.flickr.com/253/451338953_20c68143f6.jpg', 'http://farm4.static.flickr.com/3149/2592226069_2291eddfd6_o.jpg', 'https://static.flickr.com/184/424890137_b09ac2c9c4.jpg']

#@title Create the quiz session.
reward = []
sess = tf.train.SingularMonitoredSession()

counter=0

Next: let's play the game!

In [ ]:
#@title Quiz: Is this image real or fake? {run: "auto"}
#@markdown You will score +1 point if you get the answer right and -1 point if you are wrong. How many can you get right?
def count_down(time_in_seconds):
  for i in range(time_in_seconds):
    time.sleep(1)
    # print("Only {} seconds left!".format(time_in_seconds - i))



def ask_question(inputs=None):
  # Randomly choose to generate a real of fake image.
  p = np.random.uniform(low=0.0, high=1.0)
  if p > 0.3:  # Bias towards showing more fake samples.
    label = "Fake"
    # Generate a fake image.
    z = truncated_z_sample(batch_size=1, truncation=0.05)
    y = int(np.random.uniform(low=0, high=1000))
    image = sample(sess, z, y, truncation=0.5)[0]
  else:
    # Generate a real image.
    label = "Real"
    url = np.random.choice(real_image_urls)
    image = io.imread(url)

  # Keep track of the number of correct predictions.
  grid = widgets.Grid(2, 2)

  with grid.output_to(0, 0):
    plt.plot(reward, 'b*')
    plt.xlabel("Images shown across time.")
    plt.ylabel("Score.")
    plt.ylim((-1.1, 1.1))

  with grid.output_to(0, 1):
    fig=plt.imshow(image)
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.show()
    

  with grid.output_to(1, 1):
    real_or_fake_button = ipywidgets.RadioButtons(
      options=['Real', 'Fake'],
      value='Fake',
      description='This image is\n',
      disabled=False)
    display(real_or_fake_button)
  submit_button = ipywidgets.Button(description='Submit')
  display(submit_button)
  next_button = ipywidgets.Button(description='Next')

  def check_prediction(inputs=None):
    print("Checking your answer...")
    prediction = real_or_fake_button.value
    if prediction == label:
      reward.append(1)
      print(emoji.emojize('Yes! :thumbs_up: This image was {}'.format(label)))
    else:
      print(emoji.emojize('Oh no :astonished_face:, this was a {} image.'.format(label)))
      reward.append(-1)
    print("Try another one.")
    submit_button.disabled = True
    display(next_button)

  def run_again(inputs=None):
    with IPython.utils.io.capture_output(display=False):
      IPython.display.clear_output()
      ask_question()

  submit_button.on_click(check_prediction)
  next_button.on_click(run_again)

if counter < 10:
  ask_question()
else:
  print("You have reached the end of the quiz.")


## These images are created by an AI system called a Generative Model.

The AI that creates these realistic looking images is a special type of AI, called a generative model, because it generates “something”, in this case images.

<center>
<img src="https://storage.googleapis.com/dm-educational/assets/generative/artist.jpg" alt="drawing" height="220"/>
</center>







# How does this work? 




This particular generative model is called a Generative Adversarial Network because it is made up of two components that compete against each other: The Artist and the Detective.

<center>
<img src="https://storage.googleapis.com/dm-educational/assets/generative/artist_and_detective.png" alt="drawing" height="220"/>
</center>


The Artist tries to create images that look realistic. 

<center>
<img src="https://storage.googleapis.com/dm-educational/assets/generative/artist_gen.png" alt="drawing" height="220"/>
</center>


The Detective is given some real images and some images from the Artist and has to predict which ones are real or fake, just like **you** did in the quiz above.


<center>

<img src="https://storage.googleapis.com/dm-educational/assets/generative/this_looks_real.png" alt="drawing" height="240"/><img src="https://storage.googleapis.com/dm-educational/assets/generative/this_looks_fake.png" alt="drawing" height="240"/>

</center>

At the start of the game neither the Artist nor the Detective are very good. After a few rounds the Detective gets better at detecting which images from the Artist are fake. For the Artist to win the game, they have to get better too, creating more and more realistic looking images as the game goes on.

## Why are generative models important?

Throughout the centuries humans have been faced with many challenges from *\"How do we get across this river?\"* to *\"How can we cure or prevent this disease?\"*

<figure class="image">
<center>
  <img src="https://letsgetsciencey.com/wp-content/uploads/2019/03/wright_jane.jpg" alt="Dr Jane">
  <figcaption> <b>Dr. Jane C. Wright </b> pioneered research into chemotherapy treatments for cancer. </figcaption>
</center>
</figure>
 
 
Humans came up with solutions to these problems, not only by looking in text books and asking others what the answer might be, but through **creative problem solving**. Scientists and engineers worked together to imagine many possible solutions and tried them until they found a solution that worked.
 
For AI to help us solve the current and future challenges that humanity will face, it too **needs to be creative** and needs to be able to imagine new things. We will now explore this below.









# Generative models are able to exhibit some level of imagination.
 
Firstly, if a generative model creates an image of a dog, it is probably not a dog that exists in the real world, it is one that it has imagined.
 
Generative models can go one step further than just imagining a dog that does not exist, they can also imagine objects categories that do not exist, by combining object categories that do exist.

<center>
<img src="https://storage.googleapis.com/dm-educational/assets/generative/imagine_unicorn.png" alt="drawing" height="220"/>
</center>
 

## See for yourself!
In the cell below you can select two categories: **`category_A`** and **`category_B`** from the drop down menu and look at how the generative model combines these; imaging images from the selected object categories.

In [ ]:
#@title Generative models imagining new object categories by merging two existing categories.

# Taken from https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/biggan_generation_with_tf_hub.ipynb
# with minor modificiations.

num_samples = 1
num_interps = 5
truncation = 0.2
noise_seed_A = 0
category_A = "0) tench, Tinca tinca" #@param ["0) tench, Tinca tinca", "1) goldfish, Carassius auratus", "2) great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias", "3) tiger shark, Galeocerdo cuvieri", "4) hammerhead, hammerhead shark", "5) electric ray, crampfish, numbfish, torpedo", "6) stingray", "7) cock", "8) hen", "9) ostrich, Struthio camelus", "10) brambling, Fringilla montifringilla", "11) goldfinch, Carduelis carduelis", "12) house finch, linnet, Carpodacus mexicanus", "13) junco, snowbird", "14) indigo bunting, indigo finch, indigo bird, Passerina cyanea", "15) robin, American robin, Turdus migratorius", "16) bulbul", "17) jay", "18) magpie", "19) chickadee", "20) water ouzel, dipper", "21) kite", "22) bald eagle, American eagle, Haliaeetus leucocephalus", "23) vulture", "24) great grey owl, great gray owl, Strix nebulosa", "25) European fire salamander, Salamandra salamandra", "26) common newt, Triturus vulgaris", "27) eft", "28) spotted salamander, Ambystoma maculatum", "29) axolotl, mud puppy, Ambystoma mexicanum", "30) bullfrog, Rana catesbeiana", "31) tree frog, tree-frog", "32) tailed frog, bell toad, ribbed toad, tailed toad, Ascaphus trui", "33) loggerhead, loggerhead turtle, Caretta caretta", "34) leatherback turtle, leatherback, leathery turtle, Dermochelys coriacea", "35) mud turtle", "36) terrapin", "37) box turtle, box tortoise", "38) banded gecko", "39) common iguana, iguana, Iguana iguana", "40) American chameleon, anole, Anolis carolinensis", "41) whiptail, whiptail lizard", "42) agama", "43) frilled lizard, Chlamydosaurus kingi", "44) alligator lizard", "45) Gila monster, Heloderma suspectum", "46) green lizard, Lacerta viridis", "47) African chameleon, Chamaeleo chamaeleon", "48) Komodo dragon, Komodo lizard, dragon lizard, giant lizard, Varanus komodoensis", "49) African crocodile, Nile crocodile, Crocodylus niloticus", "50) American alligator, Alligator mississipiensis", "51) triceratops", "52) thunder snake, worm snake, Carphophis amoenus", "53) ringneck snake, ring-necked snake, ring snake", "54) hognose snake, puff adder, sand viper", "55) green snake, grass snake", "56) king snake, kingsnake", "57) garter snake, grass snake", "58) water snake", "59) vine snake", "60) night snake, Hypsiglena torquata", "61) boa constrictor, Constrictor constrictor", "62) rock python, rock snake, Python sebae", "63) Indian cobra, Naja naja", "64) green mamba", "65) sea snake", "66) horned viper, cerastes, sand viper, horned asp, Cerastes cornutus", "67) diamondback, diamondback rattlesnake, Crotalus adamanteus", "68) sidewinder, horned rattlesnake, Crotalus cerastes", "69) trilobite", "70) harvestman, daddy longlegs, Phalangium opilio", "71) scorpion", "72) black and gold garden spider, Argiope aurantia", "73) barn spider, Araneus cavaticus", "74) garden spider, Aranea diademata", "75) black widow, Latrodectus mactans", "76) tarantula", "77) wolf spider, hunting spider", "78) tick", "79) centipede", "80) black grouse", "81) ptarmigan", "82) ruffed grouse, partridge, Bonasa umbellus", "83) prairie chicken, prairie grouse, prairie fowl", "84) peacock", "85) quail", "86) partridge", "87) African grey, African gray, Psittacus erithacus", "88) macaw", "89) sulphur-crested cockatoo, Kakatoe galerita, Cacatua galerita", "90) lorikeet", "91) coucal", "92) bee eater", "93) hornbill", "94) hummingbird", "95) jacamar", "96) toucan", "97) drake", "98) red-breasted merganser, Mergus serrator", "99) goose", "100) black swan, Cygnus atratus", "101) tusker", "102) echidna, spiny anteater, anteater", "103) platypus, duckbill, duckbilled platypus, duck-billed platypus, Ornithorhynchus anatinus", "104) wallaby, brush kangaroo", "105) koala, koala bear, kangaroo bear, native bear, Phascolarctos cinereus", "106) wombat", "107) sea anemone, anemone", "108) brain coral", "109) flatworm, platyhelminth", "110) nematode, nematode worm, roundworm", "111) conch", "112) snail", "113) slug", "114) sea slug, nudibranch", "116) Dungeness crab, Cancer magister", "117) rock crab, Cancer irroratus", "118) fiddler crab", "119) king crab, Alaska crab, Alaskan king crab, Alaska king crab, Paralithodes camtschatica", "120) American lobster, Northern lobster, Maine lobster, Homarus americanus", "121) spiny lobster, langouste, rock lobster, crawfish, crayfish, sea crawfish", "122) crayfish, crawfish, crawdad, crawdaddy", "123) hermit crab", "124) isopod", "125) white stork, Ciconia ciconia", "126) black stork, Ciconia nigra", "127) spoonbill", "128) flamingo", "129) little blue heron, Egretta caerulea", "130) American egret, great white heron, Egretta albus", "131) bittern", "132) crane", "133) limpkin, Aramus pictus", "134) European gallinule, Porphyrio porphyrio", "135) American coot, marsh hen, mud hen, water hen, Fulica americana", "136) bustard", "137) ruddy turnstone, Arenaria interpres", "138) redshank, Tringa totanus", "139) dowitcher", "140) oystercatcher, oyster catcher", "141) pelican", "142) king penguin, Aptenodytes patagonica", "143) albatross, mollymawk", "144) grey whale, gray whale, devilfish, Eschrichtius gibbosus, Eschrichtius robustus", "145) killer whale, killer, orca, grampus, sea wolf, Orcinus orca", "146) dugong, Dugong dugon", "147) sea lion", "148) Chihuahua", "149) Japanese spaniel", "150) Maltese dog, Maltese terrier, Maltese", "151) Pekinese, Pekingese, Peke", "152) Shih-Tzu", "153) Blenheim spaniel", "154) papillon", "155) toy terrier", "156) Rhodesian ridgeback", "157) Afghan hound, Afghan", "158) basset, basset hound", "159) beagle", "160) bloodhound, sleuthhound", "161) bluetick", "162) black-and-tan coonhound", "163) Walker hound, Walker foxhound", "164) English foxhound", "165) redbone", "166) borzoi, Russian wolfhound", "167) Irish wolfhound", "168) Italian greyhound", "169) whippet", "170) Ibizan hound, Ibizan Podenco", "171) Norwegian elkhound, elkhound", "172) otterhound, otter hound", "173) Saluki, gazelle hound", "174) Scottish deerhound, deerhound", "175) Weimaraner", "176) Staffordshire bullterrier, Staffordshire bull terrier", "177) American Staffordshire terrier, Staffordshire terrier, American pit bull terrier, pit bull terrier", "178) Bedlington terrier", "179) Border terrier", "180) Kerry blue terrier", "181) Irish terrier", "182) Norfolk terrier", "183) Norwich terrier", "184) Yorkshire terrier", "185) wire-haired fox terrier", "186) Lakeland terrier", "187) Sealyham terrier, Sealyham", "188) Airedale, Airedale terrier", "189) cairn, cairn terrier", "190) Australian terrier", "191) Dandie Dinmont, Dandie Dinmont terrier", "192) Boston bull, Boston terrier", "193) miniature schnauzer", "194) giant schnauzer", "195) standard schnauzer", "196) Scotch terrier, Scottish terrier, Scottie", "197) Tibetan terrier, chrysanthemum dog", "198) silky terrier, Sydney silky", "199) soft-coated wheaten terrier", "200) West Highland white terrier", "201) Lhasa, Lhasa apso", "202) flat-coated retriever", "203) curly-coated retriever", "204) golden retriever", "205) Labrador retriever", "206) Chesapeake Bay retriever", "207) German short-haired pointer", "208) vizsla, Hungarian pointer", "209) English setter", "210) Irish setter, red setter", "211) Gordon setter", "212) Brittany spaniel", "213) clumber, clumber spaniel", "214) English springer, English springer spaniel", "215) Welsh springer spaniel", "216) cocker spaniel, English cocker spaniel, cocker", "217) Sussex spaniel", "218) Irish water spaniel", "219) kuvasz", "220) schipperke", "221) groenendael", "222) malinois", "223) briard", "224) kelpie", "225) komondor", "226) Old English sheepdog, bobtail", "227) Shetland sheepdog, Shetland sheep dog, Shetland", "228) collie", "229) Border collie", "230) Bouvier des Flandres, Bouviers des Flandres", "231) Rottweiler", "232) German shepherd, German shepherd dog, German police dog, alsatian", "233) Doberman, Doberman pinscher", "234) miniature pinscher", "235) Greater Swiss Mountain dog", "236) Bernese mountain dog", "237) Appenzeller", "238) EntleBucher", "239) boxer", "240) bull mastiff", "241) Tibetan mastiff", "242) French bulldog", "243) Great Dane", "244) Saint Bernard, St Bernard", "245) Eskimo dog, husky", "246) malamute, malemute, Alaskan malamute", "247) Siberian husky", "248) dalmatian, coach dog, carriage dog", "249) affenpinscher, monkey pinscher, monkey dog", "250) basenji", "251) pug, pug-dog", "252) Leonberg", "253) Newfoundland, Newfoundland dog", "254) Great Pyrenees", "255) Samoyed, Samoyede", "256) Pomeranian", "257) chow, chow chow", "258) keeshond", "259) Brabancon griffon", "260) Pembroke, Pembroke Welsh corgi", "261) Cardigan, Cardigan Welsh corgi", "262) toy poodle", "263) miniature poodle", "264) standard poodle", "265) Mexican hairless", "266) timber wolf, grey wolf, gray wolf, Canis lupus", "267) white wolf, Arctic wolf, Canis lupus tundrarum", "268) red wolf, maned wolf, Canis rufus, Canis niger", "269) coyote, prairie wolf, brush wolf, Canis latrans", "270) dingo, warrigal, warragal, Canis dingo", "271) dhole, Cuon alpinus", "272) African hunting dog, hyena dog, Cape hunting dog, Lycaon pictus", "273) hyena, hyaena", "274) red fox, Vulpes vulpes", "275) kit fox, Vulpes macrotis", "276) Arctic fox, white fox, Alopex lagopus", "277) grey fox, gray fox, Urocyon cinereoargenteus", "278) tabby, tabby cat", "279) tiger cat", "280) Persian cat", "281) Siamese cat, Siamese", "282) Egyptian cat", "283) cougar, puma, catamount, mountain lion, painter, panther, Felis concolor", "284) lynx, catamount", "285) leopard, Panthera pardus", "286) snow leopard, ounce, Panthera uncia", "287) jaguar, panther, Panthera onca, Felis onca", "288) lion, king of beasts, Panthera leo", "289) tiger, Panthera tigris", "290) cheetah, chetah, Acinonyx jubatus", "291) brown bear, bruin, Ursus arctos", "292) American black bear, black bear, Ursus americanus, Euarctos americanus", "293) ice bear, polar bear, Ursus Maritimus, Thalarctos maritimus", "294) sloth bear, Melursus ursinus, Ursus ursinus", "295) mongoose", "296) meerkat, mierkat", "297) tiger beetle", "298) ladybug, ladybeetle, lady beetle, ladybird, ladybird beetle", "299) ground beetle, carabid beetle", "300) long-horned beetle, longicorn, longicorn beetle", "301) leaf beetle, chrysomelid", "302) dung beetle", "303) rhinoceros beetle", "304) weevil", "305) fly", "306) bee", "307) ant, emmet, pismire", "308) grasshopper, hopper", "309) cricket", "310) walking stick, walkingstick, stick insect", "311) cockroach, roach", "312) mantis, mantid", "313) cicada, cicala", "314) leafhopper", "315) lacewing, lacewing fly", "316) dragonfly, darning needle, devil's darning needle, sewing needle, snake feeder, snake doctor, mosquito hawk, skeeter hawk", "317) damselfly", "318) admiral", "319) ringlet, ringlet butterfly", "320) monarch, monarch butterfly, milkweed butterfly, Danaus plexippus", "321) cabbage butterfly", "322) sulphur butterfly, sulfur butterfly", "323) lycaenid, lycaenid butterfly", "324) starfish, sea star", "325) sea urchin", "326) sea cucumber, holothurian", "327) wood rabbit, cottontail, cottontail rabbit", "328) hare", "329) Angora, Angora rabbit", "330) hamster", "331) porcupine, hedgehog", "332) fox squirrel, eastern fox squirrel, Sciurus niger", "333) marmot", "334) beaver", "335) guinea pig, Cavia cobaya", "336) sorrel", "337) zebra", "338) hog, pig, grunter, squealer, Sus scrofa", "339) wild boar, boar, Sus scrofa", "340) warthog", "341) hippopotamus, hippo, river horse, Hippopotamus amphibius", "342) ox", "343) water buffalo, water ox, Asiatic buffalo, Bubalus bubalis", "344) bison", "345) ram, tup", "346) bighorn, bighorn sheep, cimarron, Rocky Mountain bighorn, Rocky Mountain sheep, Ovis canadensis", "347) ibex, Capra ibex", "348) hartebeest", "349) impala, Aepyceros melampus", "350) gazelle", "351) Arabian camel, dromedary, Camelus dromedarius", "352) llama", "353) weasel", "354) mink", "355) polecat, fitch, foulmart, foumart, Mustela putorius", "356) black-footed ferret, ferret, Mustela nigripes", "357) otter", "358) skunk, polecat, wood pussy", "359) badger", "360) armadillo", "361) three-toed sloth, ai, Bradypus tridactylus", "362) orangutan, orang, orangutang, Pongo pygmaeus", "363) gorilla, Gorilla gorilla", "364) chimpanzee, chimp, Pan troglodytes", "365) gibbon, Hylobates lar", "366) siamang, Hylobates syndactylus, Symphalangus syndactylus", "367) guenon, guenon monkey", "368) patas, hussar monkey, Erythrocebus patas", "369) baboon", "370) macaque", "371) langur", "372) colobus, colobus monkey", "373) proboscis monkey, Nasalis larvatus", "374) marmoset", "375) capuchin, ringtail, Cebus capucinus", "376) howler monkey, howler", "377) titi, titi monkey", "378) spider monkey, Ateles geoffroyi", "379) squirrel monkey, Saimiri sciureus", "380) Madagascar cat, ring-tailed lemur, Lemur catta", "381) indri, indris, Indri indri, Indri brevicaudatus", "382) Indian elephant, Elephas maximus", "383) African elephant, Loxodonta africana", "384) lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens", "385) giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca", "386) barracouta, snoek", "387) eel", "388) coho, cohoe, coho salmon, blue jack, silver salmon, Oncorhynchus kisutch", "389) rock beauty, Holocanthus tricolor", "390) anemone fish", "391) sturgeon", "392) gar, garfish, garpike, billfish, Lepisosteus osseus", "393) lionfish", "394) abacus", "395) abaya", "396) academic gown, academic robe, judge's robe", "397) accordion, piano accordion, squeeze box", "398) acoustic guitar", "399) aircraft carrier, carrier, flattop, attack aircraft carrier", "400) airliner", "401) airship, dirigible", "402) altar", "403) ambulance", "404) amphibian, amphibious vehicle", "405) analog clock", "406) apiary, bee house", "407) apron", "408) ashcan, trash can, garbage can, wastebin, ash bin, ash-bin, ashbin, dustbin, trash barrel, trash bin", "409) assault rifle, assault gun", "410) backpack, back pack, knapsack, packsack, rucksack, haversack", "411) bakery, bakeshop, bakehouse", "412) balance beam, beam", "413) balloon", "414) ballpoint, ballpoint pen, ballpen, Biro", "415) Band Aid", "416) banjo", "417) bannister, banister, balustrade, balusters, handrail", "418) barbell", "419) barber chair", "420) barbershop", "421) barn", "422) barometer", "423) barrel, cask", "424) barrow, garden cart, lawn cart, wheelbarrow", "425) baseball", "426) basketball", "427) bassinet", "428) bassoon", "429) bathing cap, swimming cap", "430) bath towel", "431) bathtub, bathing tub, bath, tub", "432) beach wagon, station wagon, wagon, estate car, beach waggon, station waggon, waggon", "433) beacon, lighthouse, beacon light, pharos", "434) beaker", "435) bearskin, busby, shako", "436) beer bottle", "437) beer glass", "438) bell cote, bell cot", "439) bib", "440) bicycle-built-for-two, tandem bicycle, tandem", "441) bikini, two-piece", "442) binder, ring-binder", "443) binoculars, field glasses, opera glasses", "444) birdhouse", "445) boathouse", "446) bobsled, bobsleigh, bob", "447) bolo tie, bolo, bola tie, bola", "448) bonnet, poke bonnet", "449) bookcase", "450) bookshop, bookstore, bookstall", "451) bottlecap", "452) bow", "453) bow tie, bow-tie, bowtie", "454) brass, memorial tablet, plaque", "455) brassiere, bra, bandeau", "456) breakwater, groin, groyne, mole, bulwark, seawall, jetty", "457) breastplate, aegis, egis", "458) broom", "459) bucket, pail", "460) buckle", "461) bulletproof vest", "462) bullet train, bullet", "463) butcher shop, meat market", "465) caldron, cauldron", "466) candle, taper, wax light", "467) cannon", "468) canoe", "469) can opener, tin opener", "470) cardigan", "471) car mirror", "472) carousel, carrousel, merry-go-round, roundabout, whirligig", "473) carpenter's kit, tool kit", "474) carton", "475) car wheel", "476) cash machine, cash dispenser, automated teller machine, automatic teller machine, automated teller, automatic teller, ATM", "477) cassette", "478) cassette player", "479) castle", "480) catamaran", "481) CD player", "482) cello, violoncello", "483) cellular telephone, cellular phone, cellphone, cell, mobile phone", "484) chain", "485) chainlink fence", "486) chain mail, ring mail, mail, chain armor, chain armour, ring armor, ring armour", "487) chain saw, chainsaw", "488) chest", "489) chiffonier, commode", "490) chime, bell, gong", "491) china cabinet, china closet", "492) Christmas stocking", "493) church, church building", "494) cinema, movie theater, movie theatre, movie house, picture palace", "495) cleaver, meat cleaver, chopper", "496) cliff dwelling", "497) cloak", "498) clog, geta, patten, sabot", "499) cocktail shaker", "500) coffee mug", "501) coffeepot", "502) coil, spiral, volute, whorl, helix", "503) combination lock", "504) computer keyboard, keypad", "505) confectionery, confectionary, candy store", "506) container ship, containership, container vessel", "507) convertible", "508) corkscrew, bottle screw", "509) cornet, horn, trumpet, trump", "510) cowboy boot", "511) cowboy hat, ten-gallon hat", "512) cradle", "513) crane", "514) crash helmet", "515) crate", "516) crib, cot", "517) Crock Pot", "518) croquet ball", "519) crutch", "520) cuirass", "521) dam, dike, dyke", "522) desk", "523) desktop computer", "524) dial telephone, dial phone", "525) diaper, nappy, napkin", "526) digital clock", "527) digital watch", "528) dining table, board", "529) dishrag, dishcloth", "530) dishwasher, dish washer, dishwashing machine", "531) disk brake, disc brake", "532) dock, dockage, docking facility", "533) dogsled, dog sled, dog sleigh", "534) dome", "535) doormat, welcome mat", "536) drilling platform, offshore rig", "537) drum, membranophone, tympan", "538) drumstick", "539) dumbbell", "540) Dutch oven", "541) electric fan, blower", "542) electric guitar", "543) electric locomotive", "544) entertainment center", "545) envelope", "546) espresso maker", "547) face powder", "548) feather boa, boa", "549) file, file cabinet, filing cabinet", "550) fireboat", "551) fire engine, fire truck", "552) fire screen, fireguard", "553) flagpole, flagstaff", "554) flute, transverse flute", "555) folding chair", "556) football helmet", "557) forklift", "558) fountain", "559) fountain pen", "560) four-poster", "561) freight car", "562) French horn, horn", "563) frying pan, frypan, skillet", "564) fur coat", "565) garbage truck, dustcart", "566) gasmask, respirator, gas helmet", "567) gas pump, gasoline pump, petrol pump, island dispenser", "568) goblet", "569) go-kart", "570) golf ball", "571) golfcart, golf cart", "572) gondola", "573) gong, tam-tam", "574) gown", "575) grand piano, grand", "576) greenhouse, nursery, glasshouse", "577) grille, radiator grille", "578) grocery store, grocery, food market, market", "579) guillotine", "580) hair slide", "581) hair spray", "582) half track", "583) hammer", "584) hamper", "585) hand blower, blow dryer, blow drier, hair dryer, hair drier", "586) hand-held computer, hand-held microcomputer", "587) handkerchief, hankie, hanky, hankey", "588) hard disc, hard disk, fixed disk", "589) harmonica, mouth organ, harp, mouth harp", "590) harp", "591) harvester, reaper", "592) hatchet", "593) holster", "594) home theater, home theatre", "595) honeycomb", "596) hook, claw", "597) hoopskirt, crinoline", "598) horizontal bar, high bar", "599) horse cart, horse-cart", "600) hourglass", "601) iPod", "602) iron, smoothing iron", "603) jack-o'-lantern", "604) jean, blue jean, denim", "605) jeep, landrover", "606) jersey, T-shirt, tee shirt", "607) jigsaw puzzle", "608) jinrikisha, ricksha, rickshaw", "609) joystick", "610) kimono", "611) knee pad", "612) knot", "613) lab coat, laboratory coat", "614) ladle", "615) lampshade, lamp shade", "616) laptop, laptop computer", "617) lawn mower, mower", "618) lens cap, lens cover", "619) letter opener, paper knife, paperknife", "620) library", "621) lifeboat", "622) lighter, light, igniter, ignitor", "623) limousine, limo", "624) liner, ocean liner", "625) lipstick, lip rouge", "626) Loafer", "627) lotion", "628) loudspeaker, speaker, speaker unit, loudspeaker system, speaker system", "629) loupe, jeweler's loupe", "630) lumbermill, sawmill", "631) magnetic compass", "632) mailbag, postbag", "633) mailbox, letter box", "634) maillot", "635) maillot, tank suit", "636) manhole cover", "637) maraca", "638) marimba, xylophone", "639) mask", "640) matchstick", "641) maypole", "642) maze, labyrinth", "643) measuring cup", "644) medicine chest, medicine cabinet", "645) megalith, megalithic structure", "646) microphone, mike", "647) microwave, microwave oven", "648) military uniform", "649) milk can", "650) minibus", "651) miniskirt, mini", "652) minivan", "653) missile", "654) mitten", "655) mixing bowl", "656) mobile home, manufactured home", "657) Model T", "658) modem", "659) monastery", "660) monitor", "661) moped", "662) mortar", "663) mortarboard", "664) mosque", "665) mosquito net", "666) motor scooter, scooter", "667) mountain bike, all-terrain bike, off-roader", "668) mountain tent", "669) mouse, computer mouse", "670) mousetrap", "671) moving van", "672) muzzle", "673) nail", "674) neck brace", "675) necklace", "676) nipple", "677) notebook, notebook computer", "678) obelisk", "679) oboe, hautboy, hautbois", "680) ocarina, sweet potato", "681) odometer, hodometer, mileometer, milometer", "682) oil filter", "683) organ, pipe organ", "684) oscilloscope, scope, cathode-ray oscilloscope, CRO", "685) overskirt", "686) oxcart", "687) oxygen mask", "688) packet", "689) paddle, boat paddle", "690) paddlewheel, paddle wheel", "691) padlock", "692) paintbrush", "693) pajama, pyjama, pj's, jammies", "694) palace", "695) panpipe, pandean pipe, syrinx", "696) paper towel", "697) parachute, chute", "698) parallel bars, bars", "699) park bench", "700) parking meter", "701) passenger car, coach, carriage", "702) patio, terrace", "703) pay-phone, pay-station", "704) pedestal, plinth, footstall", "705) pencil box, pencil case", "706) pencil sharpener", "707) perfume, essence", "708) Petri dish", "709) photocopier", "710) pick, plectrum, plectron", "711) pickelhaube", "712) picket fence, paling", "713) pickup, pickup truck", "714) pier", "715) piggy bank, penny bank", "716) pill bottle", "717) pillow", "718) ping-pong ball", "719) pinwheel", "720) pirate, pirate ship", "721) pitcher, ewer", "722) plane, carpenter's plane, woodworking plane", "723) planetarium", "724) plastic bag", "725) plate rack", "726) plow, plough", "727) plunger, plumber's helper", "728) Polaroid camera, Polaroid Land camera", "729) pole", "730) police van, police wagon, paddy wagon, patrol wagon, wagon, black Maria", "731) poncho", "732) pool table, billiard table, snooker table", "733) pop bottle, soda bottle", "734) pot, flowerpot", "735) potter's wheel", "736) power drill", "737) prayer rug, prayer mat", "738) printer", "739) prison, prison house", "740) projectile, missile", "741) projector", "742) puck, hockey puck", "743) punching bag, punch bag, punching ball, punchball", "744) purse", "745) quill, quill pen", "746) quilt, comforter, comfort, puff", "747) racer, race car, racing car", "748) racket, racquet", "749) radiator", "750) radio, wireless", "751) radio telescope, radio reflector", "752) rain barrel", "753) recreational vehicle, RV, R.V.", "754) reel", "755) reflex camera", "756) refrigerator, icebox", "757) remote control, remote", "758) restaurant, eating house, eating place, eatery", "759) revolver, six-gun, six-shooter", "760) rifle", "761) rocking chair, rocker", "762) rotisserie", "763) rubber eraser, rubber, pencil eraser", "764) rugby ball", "765) rule, ruler", "766) running shoe", "767) safe", "768) safety pin", "769) saltshaker, salt shaker", "770) sandal", "771) sarong", "772) sax, saxophone", "773) scabbard", "774) scale, weighing machine", "775) school bus", "776) schooner", "777) scoreboard", "778) screen, CRT screen", "779) screw", "780) screwdriver", "781) seat belt, seatbelt", "782) sewing machine", "783) shield, buckler", "784) shoe shop, shoe-shop, shoe store", "785) shoji", "786) shopping basket", "787) shopping cart", "788) shovel", "789) shower cap", "790) shower curtain", "791) ski", "792) ski mask", "793) sleeping bag", "794) slide rule, slipstick", "795) sliding door", "796) slot, one-armed bandit", "797) snorkel", "798) snowmobile", "799) snowplow, snowplough", "800) soap dispenser", "801) soccer ball", "802) sock", "803) solar dish, solar collector, solar furnace", "804) sombrero", "805) soup bowl", "806) space bar", "807) space heater", "808) space shuttle", "809) spatula", "810) speedboat", "811) spider web, spider's web", "812) spindle", "813) sports car, sport car", "814) spotlight, spot", "815) stage", "816) steam locomotive", "817) steel arch bridge", "818) steel drum", "819) stethoscope", "820) stole", "821) stone wall", "822) stopwatch, stop watch", "823) stove", "824) strainer", "825) streetcar, tram, tramcar, trolley, trolley car", "826) stretcher", "827) studio couch, day bed", "828) stupa, tope", "829) submarine, pigboat, sub, U-boat", "830) suit, suit of clothes", "831) sundial", "832) sunglass", "833) sunglasses, dark glasses, shades", "834) sunscreen, sunblock, sun blocker", "835) suspension bridge", "836) swab, swob, mop", "837) sweatshirt", "838) swimming trunks, bathing trunks", "839) swing", "840) switch, electric switch, electrical switch", "841) syringe", "842) table lamp", "843) tank, army tank, armored combat vehicle, armoured combat vehicle", "844) tape player", "845) teapot", "846) teddy, teddy bear", "847) television, television system", "848) tennis ball", "849) thatch, thatched roof", "850) theater curtain, theatre curtain", "851) thimble", "852) thresher, thrasher, threshing machine", "853) throne", "854) tile roof", "855) toaster", "856) tobacco shop, tobacconist shop, tobacconist", "857) toilet seat", "858) torch", "859) totem pole", "860) tow truck, tow car, wrecker", "861) toyshop", "862) tractor", "863) trailer truck, tractor trailer, trucking rig, rig, articulated lorry, semi", "864) tray", "865) trench coat", "866) tricycle, trike, velocipede", "867) trimaran", "868) tripod", "869) triumphal arch", "870) trolleybus, trolley coach, trackless trolley", "871) trombone", "872) tub, vat", "873) turnstile", "874) typewriter keyboard", "875) umbrella", "876) unicycle, monocycle", "877) upright, upright piano", "878) vacuum, vacuum cleaner", "879) vase", "880) vault", "881) velvet", "882) vending machine", "883) vestment", "884) viaduct", "885) violin, fiddle", "886) volleyball", "887) waffle iron", "888) wall clock", "889) wallet, billfold, notecase, pocketbook", "890) wardrobe, closet, press", "891) warplane, military plane", "892) washbasin, handbasin, washbowl, lavabo, wash-hand basin", "893) washer, automatic washer, washing machine", "894) water bottle", "895) water jug", "896) water tower", "897) whiskey jug", "898) whistle", "899) wig", "900) window screen", "901) window shade", "902) Windsor tie", "903) wine bottle", "904) wing", "905) wok", "906) wooden spoon", "907) wool, woolen, woollen", "908) worm fence, snake fence, snake-rail fence, Virginia fence", "909) wreck", "910) yawl", "911) yurt", "912) web site, website, internet site, site", "913) comic book", "914) crossword puzzle, crossword", "915) street sign", "916) traffic light, traffic signal, stoplight", "917) book jacket, dust cover, dust jacket, dust wrapper", "918) menu", "919) plate", "920) guacamole", "921) consomme", "922) hot pot, hotpot", "923) trifle", "924) ice cream, icecream", "925) ice lolly, lolly, lollipop, popsicle", "926) French loaf", "927) bagel, beigel", "928) pretzel", "929) cheeseburger", "930) hotdog, hot dog, red hot", "931) mashed potato", "932) head cabbage", "933) broccoli", "934) cauliflower", "935) zucchini, courgette", "936) spaghetti squash", "937) acorn squash", "938) butternut squash", "939) cucumber, cuke", "940) artichoke, globe artichoke", "941) bell pepper", "942) cardoon", "943) mushroom", "944) Granny Smith", "945) strawberry", "946) orange", "947) lemon", "948) fig", "949) pineapple, ananas", "950) banana", "951) jackfruit, jak, jack", "952) custard apple", "953) pomegranate", "954) hay", "955) carbonara", "956) chocolate sauce, chocolate syrup", "957) dough", "958) meat loaf, meatloaf", "959) pizza, pizza pie", "960) potpie", "961) burrito", "962) red wine", "963) espresso", "964) cup", "965) eggnog", "966) alp", "967) bubble", "968) cliff, drop, drop-off", "969) coral reef", "970) geyser", "971) lakeside, lakeshore", "972) promontory, headland, head, foreland", "973) sandbar, sand bar", "974) seashore, coast, seacoast, sea-coast", "975) valley, vale", "976) volcano", "977) ballplayer, baseball player", "978) groom, bridegroom", "979) scuba diver", "980) rapeseed", "981) daisy", "982) yellow lady's slipper, yellow lady-slipper, Cypripedium calceolus, Cypripedium parviflorum", "983) corn", "984) acorn", "985) hip, rose hip, rosehip", "986) buckeye, horse chestnut, conker", "987) coral fungus", "988) agaric", "989) gyromitra", "990) stinkhorn, carrion fungus", "991) earthstar", "993) bolete", "994) ear, spike, capitulum", "995) toilet tissue, toilet paper, bathroom tissue"]
noise_seed_B = 0
category_B = "0) tench, Tinca tinca" #@param ["0) tench, Tinca tinca", "1) goldfish, Carassius auratus", "2) great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias", "3) tiger shark, Galeocerdo cuvieri", "4) hammerhead, hammerhead shark", "5) electric ray, crampfish, numbfish, torpedo", "6) stingray", "7) cock", "8) hen", "9) ostrich, Struthio camelus", "10) brambling, Fringilla montifringilla", "11) goldfinch, Carduelis carduelis", "12) house finch, linnet, Carpodacus mexicanus", "13) junco, snowbird", "14) indigo bunting, indigo finch, indigo bird, Passerina cyanea", "15) robin, American robin, Turdus migratorius", "16) bulbul", "17) jay", "18) magpie", "19) chickadee", "20) water ouzel, dipper", "21) kite", "22) bald eagle, American eagle, Haliaeetus leucocephalus", "23) vulture", "24) great grey owl, great gray owl, Strix nebulosa", "25) European fire salamander, Salamandra salamandra", "26) common newt, Triturus vulgaris", "27) eft", "28) spotted salamander, Ambystoma maculatum", "29) axolotl, mud puppy, Ambystoma mexicanum", "30) bullfrog, Rana catesbeiana", "31) tree frog, tree-frog", "32) tailed frog, bell toad, ribbed toad, tailed toad, Ascaphus trui", "33) loggerhead, loggerhead turtle, Caretta caretta", "34) leatherback turtle, leatherback, leathery turtle, Dermochelys coriacea", "35) mud turtle", "36) terrapin", "37) box turtle, box tortoise", "38) banded gecko", "39) common iguana, iguana, Iguana iguana", "40) American chameleon, anole, Anolis carolinensis", "41) whiptail, whiptail lizard", "42) agama", "43) frilled lizard, Chlamydosaurus kingi", "44) alligator lizard", "45) Gila monster, Heloderma suspectum", "46) green lizard, Lacerta viridis", "47) African chameleon, Chamaeleo chamaeleon", "48) Komodo dragon, Komodo lizard, dragon lizard, giant lizard, Varanus komodoensis", "49) African crocodile, Nile crocodile, Crocodylus niloticus", "50) American alligator, Alligator mississipiensis", "51) triceratops", "52) thunder snake, worm snake, Carphophis amoenus", "53) ringneck snake, ring-necked snake, ring snake", "54) hognose snake, puff adder, sand viper", "55) green snake, grass snake", "56) king snake, kingsnake", "57) garter snake, grass snake", "58) water snake", "59) vine snake", "60) night snake, Hypsiglena torquata", "61) boa constrictor, Constrictor constrictor", "62) rock python, rock snake, Python sebae", "63) Indian cobra, Naja naja", "64) green mamba", "65) sea snake", "66) horned viper, cerastes, sand viper, horned asp, Cerastes cornutus", "67) diamondback, diamondback rattlesnake, Crotalus adamanteus", "68) sidewinder, horned rattlesnake, Crotalus cerastes", "69) trilobite", "70) harvestman, daddy longlegs, Phalangium opilio", "71) scorpion", "72) black and gold garden spider, Argiope aurantia", "73) barn spider, Araneus cavaticus", "74) garden spider, Aranea diademata", "75) black widow, Latrodectus mactans", "76) tarantula", "77) wolf spider, hunting spider", "78) tick", "79) centipede", "80) black grouse", "81) ptarmigan", "82) ruffed grouse, partridge, Bonasa umbellus", "83) prairie chicken, prairie grouse, prairie fowl", "84) peacock", "85) quail", "86) partridge", "87) African grey, African gray, Psittacus erithacus", "88) macaw", "89) sulphur-crested cockatoo, Kakatoe galerita, Cacatua galerita", "90) lorikeet", "91) coucal", "92) bee eater", "93) hornbill", "94) hummingbird", "95) jacamar", "96) toucan", "97) drake", "98) red-breasted merganser, Mergus serrator", "99) goose", "100) black swan, Cygnus atratus", "101) tusker", "102) echidna, spiny anteater, anteater", "103) platypus, duckbill, duckbilled platypus, duck-billed platypus, Ornithorhynchus anatinus", "104) wallaby, brush kangaroo", "105) koala, koala bear, kangaroo bear, native bear, Phascolarctos cinereus", "106) wombat", "107) sea anemone, anemone", "108) brain coral", "109) flatworm, platyhelminth", "110) nematode, nematode worm, roundworm", "111) conch", "112) snail", "113) slug", "114) sea slug, nudibranch", "116) Dungeness crab, Cancer magister", "117) rock crab, Cancer irroratus", "118) fiddler crab", "119) king crab, Alaska crab, Alaskan king crab, Alaska king crab, Paralithodes camtschatica", "120) American lobster, Northern lobster, Maine lobster, Homarus americanus", "121) spiny lobster, langouste, rock lobster, crawfish, crayfish, sea crawfish", "122) crayfish, crawfish, crawdad, crawdaddy", "123) hermit crab", "124) isopod", "125) white stork, Ciconia ciconia", "126) black stork, Ciconia nigra", "127) spoonbill", "128) flamingo", "129) little blue heron, Egretta caerulea", "130) American egret, great white heron, Egretta albus", "131) bittern", "132) crane", "133) limpkin, Aramus pictus", "134) European gallinule, Porphyrio porphyrio", "135) American coot, marsh hen, mud hen, water hen, Fulica americana", "136) bustard", "137) ruddy turnstone, Arenaria interpres", "138) redshank, Tringa totanus", "139) dowitcher", "140) oystercatcher, oyster catcher", "141) pelican", "142) king penguin, Aptenodytes patagonica", "143) albatross, mollymawk", "144) grey whale, gray whale, devilfish, Eschrichtius gibbosus, Eschrichtius robustus", "145) killer whale, killer, orca, grampus, sea wolf, Orcinus orca", "146) dugong, Dugong dugon", "147) sea lion", "148) Chihuahua", "149) Japanese spaniel", "150) Maltese dog, Maltese terrier, Maltese", "151) Pekinese, Pekingese, Peke", "152) Shih-Tzu", "153) Blenheim spaniel", "154) papillon", "155) toy terrier", "156) Rhodesian ridgeback", "157) Afghan hound, Afghan", "158) basset, basset hound", "159) beagle", "160) bloodhound, sleuthhound", "161) bluetick", "162) black-and-tan coonhound", "163) Walker hound, Walker foxhound", "164) English foxhound", "165) redbone", "166) borzoi, Russian wolfhound", "167) Irish wolfhound", "168) Italian greyhound", "169) whippet", "170) Ibizan hound, Ibizan Podenco", "171) Norwegian elkhound, elkhound", "172) otterhound, otter hound", "173) Saluki, gazelle hound", "174) Scottish deerhound, deerhound", "175) Weimaraner", "176) Staffordshire bullterrier, Staffordshire bull terrier", "177) American Staffordshire terrier, Staffordshire terrier, American pit bull terrier, pit bull terrier", "178) Bedlington terrier", "179) Border terrier", "180) Kerry blue terrier", "181) Irish terrier", "182) Norfolk terrier", "183) Norwich terrier", "184) Yorkshire terrier", "185) wire-haired fox terrier", "186) Lakeland terrier", "187) Sealyham terrier, Sealyham", "188) Airedale, Airedale terrier", "189) cairn, cairn terrier", "190) Australian terrier", "191) Dandie Dinmont, Dandie Dinmont terrier", "192) Boston bull, Boston terrier", "193) miniature schnauzer", "194) giant schnauzer", "195) standard schnauzer", "196) Scotch terrier, Scottish terrier, Scottie", "197) Tibetan terrier, chrysanthemum dog", "198) silky terrier, Sydney silky", "199) soft-coated wheaten terrier", "200) West Highland white terrier", "201) Lhasa, Lhasa apso", "202) flat-coated retriever", "203) curly-coated retriever", "204) golden retriever", "205) Labrador retriever", "206) Chesapeake Bay retriever", "207) German short-haired pointer", "208) vizsla, Hungarian pointer", "209) English setter", "210) Irish setter, red setter", "211) Gordon setter", "212) Brittany spaniel", "213) clumber, clumber spaniel", "214) English springer, English springer spaniel", "215) Welsh springer spaniel", "216) cocker spaniel, English cocker spaniel, cocker", "217) Sussex spaniel", "218) Irish water spaniel", "219) kuvasz", "220) schipperke", "221) groenendael", "222) malinois", "223) briard", "224) kelpie", "225) komondor", "226) Old English sheepdog, bobtail", "227) Shetland sheepdog, Shetland sheep dog, Shetland", "228) collie", "229) Border collie", "230) Bouvier des Flandres, Bouviers des Flandres", "231) Rottweiler", "232) German shepherd, German shepherd dog, German police dog, alsatian", "233) Doberman, Doberman pinscher", "234) miniature pinscher", "235) Greater Swiss Mountain dog", "236) Bernese mountain dog", "237) Appenzeller", "238) EntleBucher", "239) boxer", "240) bull mastiff", "241) Tibetan mastiff", "242) French bulldog", "243) Great Dane", "244) Saint Bernard, St Bernard", "245) Eskimo dog, husky", "246) malamute, malemute, Alaskan malamute", "247) Siberian husky", "248) dalmatian, coach dog, carriage dog", "249) affenpinscher, monkey pinscher, monkey dog", "250) basenji", "251) pug, pug-dog", "252) Leonberg", "253) Newfoundland, Newfoundland dog", "254) Great Pyrenees", "255) Samoyed, Samoyede", "256) Pomeranian", "257) chow, chow chow", "258) keeshond", "259) Brabancon griffon", "260) Pembroke, Pembroke Welsh corgi", "261) Cardigan, Cardigan Welsh corgi", "262) toy poodle", "263) miniature poodle", "264) standard poodle", "265) Mexican hairless", "266) timber wolf, grey wolf, gray wolf, Canis lupus", "267) white wolf, Arctic wolf, Canis lupus tundrarum", "268) red wolf, maned wolf, Canis rufus, Canis niger", "269) coyote, prairie wolf, brush wolf, Canis latrans", "270) dingo, warrigal, warragal, Canis dingo", "271) dhole, Cuon alpinus", "272) African hunting dog, hyena dog, Cape hunting dog, Lycaon pictus", "273) hyena, hyaena", "274) red fox, Vulpes vulpes", "275) kit fox, Vulpes macrotis", "276) Arctic fox, white fox, Alopex lagopus", "277) grey fox, gray fox, Urocyon cinereoargenteus", "278) tabby, tabby cat", "279) tiger cat", "280) Persian cat", "281) Siamese cat, Siamese", "282) Egyptian cat", "283) cougar, puma, catamount, mountain lion, painter, panther, Felis concolor", "284) lynx, catamount", "285) leopard, Panthera pardus", "286) snow leopard, ounce, Panthera uncia", "287) jaguar, panther, Panthera onca, Felis onca", "288) lion, king of beasts, Panthera leo", "289) tiger, Panthera tigris", "290) cheetah, chetah, Acinonyx jubatus", "291) brown bear, bruin, Ursus arctos", "292) American black bear, black bear, Ursus americanus, Euarctos americanus", "293) ice bear, polar bear, Ursus Maritimus, Thalarctos maritimus", "294) sloth bear, Melursus ursinus, Ursus ursinus", "295) mongoose", "296) meerkat, mierkat", "297) tiger beetle", "298) ladybug, ladybeetle, lady beetle, ladybird, ladybird beetle", "299) ground beetle, carabid beetle", "300) long-horned beetle, longicorn, longicorn beetle", "301) leaf beetle, chrysomelid", "302) dung beetle", "303) rhinoceros beetle", "304) weevil", "305) fly", "306) bee", "307) ant, emmet, pismire", "308) grasshopper, hopper", "309) cricket", "310) walking stick, walkingstick, stick insect", "311) cockroach, roach", "312) mantis, mantid", "313) cicada, cicala", "314) leafhopper", "315) lacewing, lacewing fly", "316) dragonfly, darning needle, devil's darning needle, sewing needle, snake feeder, snake doctor, mosquito hawk, skeeter hawk", "317) damselfly", "318) admiral", "319) ringlet, ringlet butterfly", "320) monarch, monarch butterfly, milkweed butterfly, Danaus plexippus", "321) cabbage butterfly", "322) sulphur butterfly, sulfur butterfly", "323) lycaenid, lycaenid butterfly", "324) starfish, sea star", "325) sea urchin", "326) sea cucumber, holothurian", "327) wood rabbit, cottontail, cottontail rabbit", "328) hare", "329) Angora, Angora rabbit", "330) hamster", "331) porcupine, hedgehog", "332) fox squirrel, eastern fox squirrel, Sciurus niger", "333) marmot", "334) beaver", "335) guinea pig, Cavia cobaya", "336) sorrel", "337) zebra", "338) hog, pig, grunter, squealer, Sus scrofa", "339) wild boar, boar, Sus scrofa", "340) warthog", "341) hippopotamus, hippo, river horse, Hippopotamus amphibius", "342) ox", "343) water buffalo, water ox, Asiatic buffalo, Bubalus bubalis", "344) bison", "345) ram, tup", "346) bighorn, bighorn sheep, cimarron, Rocky Mountain bighorn, Rocky Mountain sheep, Ovis canadensis", "347) ibex, Capra ibex", "348) hartebeest", "349) impala, Aepyceros melampus", "350) gazelle", "351) Arabian camel, dromedary, Camelus dromedarius", "352) llama", "353) weasel", "354) mink", "355) polecat, fitch, foulmart, foumart, Mustela putorius", "356) black-footed ferret, ferret, Mustela nigripes", "357) otter", "358) skunk, polecat, wood pussy", "359) badger", "360) armadillo", "361) three-toed sloth, ai, Bradypus tridactylus", "362) orangutan, orang, orangutang, Pongo pygmaeus", "363) gorilla, Gorilla gorilla", "364) chimpanzee, chimp, Pan troglodytes", "365) gibbon, Hylobates lar", "366) siamang, Hylobates syndactylus, Symphalangus syndactylus", "367) guenon, guenon monkey", "368) patas, hussar monkey, Erythrocebus patas", "369) baboon", "370) macaque", "371) langur", "372) colobus, colobus monkey", "373) proboscis monkey, Nasalis larvatus", "374) marmoset", "375) capuchin, ringtail, Cebus capucinus", "376) howler monkey, howler", "377) titi, titi monkey", "378) spider monkey, Ateles geoffroyi", "379) squirrel monkey, Saimiri sciureus", "380) Madagascar cat, ring-tailed lemur, Lemur catta", "381) indri, indris, Indri indri, Indri brevicaudatus", "382) Indian elephant, Elephas maximus", "383) African elephant, Loxodonta africana", "384) lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens", "385) giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca", "386) barracouta, snoek", "387) eel", "388) coho, cohoe, coho salmon, blue jack, silver salmon, Oncorhynchus kisutch", "389) rock beauty, Holocanthus tricolor", "390) anemone fish", "391) sturgeon", "392) gar, garfish, garpike, billfish, Lepisosteus osseus", "393) lionfish", "394) abacus", "395) abaya", "396) academic gown, academic robe, judge's robe", "397) accordion, piano accordion, squeeze box", "398) acoustic guitar", "399) aircraft carrier, carrier, flattop, attack aircraft carrier", "400) airliner", "401) airship, dirigible", "402) altar", "403) ambulance", "404) amphibian, amphibious vehicle", "405) analog clock", "406) apiary, bee house", "407) apron", "408) ashcan, trash can, garbage can, wastebin, ash bin, ash-bin, ashbin, dustbin, trash barrel, trash bin", "409) assault rifle, assault gun", "410) backpack, back pack, knapsack, packsack, rucksack, haversack", "411) bakery, bakeshop, bakehouse", "412) balance beam, beam", "413) balloon", "414) ballpoint, ballpoint pen, ballpen, Biro", "415) Band Aid", "416) banjo", "417) bannister, banister, balustrade, balusters, handrail", "418) barbell", "419) barber chair", "420) barbershop", "421) barn", "422) barometer", "423) barrel, cask", "424) barrow, garden cart, lawn cart, wheelbarrow", "425) baseball", "426) basketball", "427) bassinet", "428) bassoon", "429) bathing cap, swimming cap", "430) bath towel", "431) bathtub, bathing tub, bath, tub", "432) beach wagon, station wagon, wagon, estate car, beach waggon, station waggon, waggon", "433) beacon, lighthouse, beacon light, pharos", "434) beaker", "435) bearskin, busby, shako", "436) beer bottle", "437) beer glass", "438) bell cote, bell cot", "439) bib", "440) bicycle-built-for-two, tandem bicycle, tandem", "441) bikini, two-piece", "442) binder, ring-binder", "443) binoculars, field glasses, opera glasses", "444) birdhouse", "445) boathouse", "446) bobsled, bobsleigh, bob", "447) bolo tie, bolo, bola tie, bola", "448) bonnet, poke bonnet", "449) bookcase", "450) bookshop, bookstore, bookstall", "451) bottlecap", "452) bow", "453) bow tie, bow-tie, bowtie", "454) brass, memorial tablet, plaque", "455) brassiere, bra, bandeau", "456) breakwater, groin, groyne, mole, bulwark, seawall, jetty", "457) breastplate, aegis, egis", "458) broom", "459) bucket, pail", "460) buckle", "461) bulletproof vest", "462) bullet train, bullet", "463) butcher shop, meat market", "465) caldron, cauldron", "466) candle, taper, wax light", "467) cannon", "468) canoe", "469) can opener, tin opener", "470) cardigan", "471) car mirror", "472) carousel, carrousel, merry-go-round, roundabout, whirligig", "473) carpenter's kit, tool kit", "474) carton", "475) car wheel", "476) cash machine, cash dispenser, automated teller machine, automatic teller machine, automated teller, automatic teller, ATM", "477) cassette", "478) cassette player", "479) castle", "480) catamaran", "481) CD player", "482) cello, violoncello", "483) cellular telephone, cellular phone, cellphone, cell, mobile phone", "484) chain", "485) chainlink fence", "486) chain mail, ring mail, mail, chain armor, chain armour, ring armor, ring armour", "487) chain saw, chainsaw", "488) chest", "489) chiffonier, commode", "490) chime, bell, gong", "491) china cabinet, china closet", "492) Christmas stocking", "493) church, church building", "494) cinema, movie theater, movie theatre, movie house, picture palace", "495) cleaver, meat cleaver, chopper", "496) cliff dwelling", "497) cloak", "498) clog, geta, patten, sabot", "499) cocktail shaker", "500) coffee mug", "501) coffeepot", "502) coil, spiral, volute, whorl, helix", "503) combination lock", "504) computer keyboard, keypad", "505) confectionery, confectionary, candy store", "506) container ship, containership, container vessel", "507) convertible", "508) corkscrew, bottle screw", "509) cornet, horn, trumpet, trump", "510) cowboy boot", "511) cowboy hat, ten-gallon hat", "512) cradle", "513) crane", "514) crash helmet", "515) crate", "516) crib, cot", "517) Crock Pot", "518) croquet ball", "519) crutch", "520) cuirass", "521) dam, dike, dyke", "522) desk", "523) desktop computer", "524) dial telephone, dial phone", "525) diaper, nappy, napkin", "526) digital clock", "527) digital watch", "528) dining table, board", "529) dishrag, dishcloth", "530) dishwasher, dish washer, dishwashing machine", "531) disk brake, disc brake", "532) dock, dockage, docking facility", "533) dogsled, dog sled, dog sleigh", "534) dome", "535) doormat, welcome mat", "536) drilling platform, offshore rig", "537) drum, membranophone, tympan", "538) drumstick", "539) dumbbell", "540) Dutch oven", "541) electric fan, blower", "542) electric guitar", "543) electric locomotive", "544) entertainment center", "545) envelope", "546) espresso maker", "547) face powder", "548) feather boa, boa", "549) file, file cabinet, filing cabinet", "550) fireboat", "551) fire engine, fire truck", "552) fire screen, fireguard", "553) flagpole, flagstaff", "554) flute, transverse flute", "555) folding chair", "556) football helmet", "557) forklift", "558) fountain", "559) fountain pen", "560) four-poster", "561) freight car", "562) French horn, horn", "563) frying pan, frypan, skillet", "564) fur coat", "565) garbage truck, dustcart", "566) gasmask, respirator, gas helmet", "567) gas pump, gasoline pump, petrol pump, island dispenser", "568) goblet", "569) go-kart", "570) golf ball", "571) golfcart, golf cart", "572) gondola", "573) gong, tam-tam", "574) gown", "575) grand piano, grand", "576) greenhouse, nursery, glasshouse", "577) grille, radiator grille", "578) grocery store, grocery, food market, market", "579) guillotine", "580) hair slide", "581) hair spray", "582) half track", "583) hammer", "584) hamper", "585) hand blower, blow dryer, blow drier, hair dryer, hair drier", "586) hand-held computer, hand-held microcomputer", "587) handkerchief, hankie, hanky, hankey", "588) hard disc, hard disk, fixed disk", "589) harmonica, mouth organ, harp, mouth harp", "590) harp", "591) harvester, reaper", "592) hatchet", "593) holster", "594) home theater, home theatre", "595) honeycomb", "596) hook, claw", "597) hoopskirt, crinoline", "598) horizontal bar, high bar", "599) horse cart, horse-cart", "600) hourglass", "601) iPod", "602) iron, smoothing iron", "603) jack-o'-lantern", "604) jean, blue jean, denim", "605) jeep, landrover", "606) jersey, T-shirt, tee shirt", "607) jigsaw puzzle", "608) jinrikisha, ricksha, rickshaw", "609) joystick", "610) kimono", "611) knee pad", "612) knot", "613) lab coat, laboratory coat", "614) ladle", "615) lampshade, lamp shade", "616) laptop, laptop computer", "617) lawn mower, mower", "618) lens cap, lens cover", "619) letter opener, paper knife, paperknife", "620) library", "621) lifeboat", "622) lighter, light, igniter, ignitor", "623) limousine, limo", "624) liner, ocean liner", "625) lipstick, lip rouge", "626) Loafer", "627) lotion", "628) loudspeaker, speaker, speaker unit, loudspeaker system, speaker system", "629) loupe, jeweler's loupe", "630) lumbermill, sawmill", "631) magnetic compass", "632) mailbag, postbag", "633) mailbox, letter box", "634) maillot", "635) maillot, tank suit", "636) manhole cover", "637) maraca", "638) marimba, xylophone", "639) mask", "640) matchstick", "641) maypole", "642) maze, labyrinth", "643) measuring cup", "644) medicine chest, medicine cabinet", "645) megalith, megalithic structure", "646) microphone, mike", "647) microwave, microwave oven", "648) military uniform", "649) milk can", "650) minibus", "651) miniskirt, mini", "652) minivan", "653) missile", "654) mitten", "655) mixing bowl", "656) mobile home, manufactured home", "657) Model T", "658) modem", "659) monastery", "660) monitor", "661) moped", "662) mortar", "663) mortarboard", "664) mosque", "665) mosquito net", "666) motor scooter, scooter", "667) mountain bike, all-terrain bike, off-roader", "668) mountain tent", "669) mouse, computer mouse", "670) mousetrap", "671) moving van", "672) muzzle", "673) nail", "674) neck brace", "675) necklace", "676) nipple", "677) notebook, notebook computer", "678) obelisk", "679) oboe, hautboy, hautbois", "680) ocarina, sweet potato", "681) odometer, hodometer, mileometer, milometer", "682) oil filter", "683) organ, pipe organ", "684) oscilloscope, scope, cathode-ray oscilloscope, CRO", "685) overskirt", "686) oxcart", "687) oxygen mask", "688) packet", "689) paddle, boat paddle", "690) paddlewheel, paddle wheel", "691) padlock", "692) paintbrush", "693) pajama, pyjama, pj's, jammies", "694) palace", "695) panpipe, pandean pipe, syrinx", "696) paper towel", "697) parachute, chute", "698) parallel bars, bars", "699) park bench", "700) parking meter", "701) passenger car, coach, carriage", "702) patio, terrace", "703) pay-phone, pay-station", "704) pedestal, plinth, footstall", "705) pencil box, pencil case", "706) pencil sharpener", "707) perfume, essence", "708) Petri dish", "709) photocopier", "710) pick, plectrum, plectron", "711) pickelhaube", "712) picket fence, paling", "713) pickup, pickup truck", "714) pier", "715) piggy bank, penny bank", "716) pill bottle", "717) pillow", "718) ping-pong ball", "719) pinwheel", "720) pirate, pirate ship", "721) pitcher, ewer", "722) plane, carpenter's plane, woodworking plane", "723) planetarium", "724) plastic bag", "725) plate rack", "726) plow, plough", "727) plunger, plumber's helper", "728) Polaroid camera, Polaroid Land camera", "729) pole", "730) police van, police wagon, paddy wagon, patrol wagon, wagon, black Maria", "731) poncho", "732) pool table, billiard table, snooker table", "733) pop bottle, soda bottle", "734) pot, flowerpot", "735) potter's wheel", "736) power drill", "737) prayer rug, prayer mat", "738) printer", "739) prison, prison house", "740) projectile, missile", "741) projector", "742) puck, hockey puck", "743) punching bag, punch bag, punching ball, punchball", "744) purse", "745) quill, quill pen", "746) quilt, comforter, comfort, puff", "747) racer, race car, racing car", "748) racket, racquet", "749) radiator", "750) radio, wireless", "751) radio telescope, radio reflector", "752) rain barrel", "753) recreational vehicle, RV, R.V.", "754) reel", "755) reflex camera", "756) refrigerator, icebox", "757) remote control, remote", "758) restaurant, eating house, eating place, eatery", "759) revolver, six-gun, six-shooter", "760) rifle", "761) rocking chair, rocker", "762) rotisserie", "763) rubber eraser, rubber, pencil eraser", "764) rugby ball", "765) rule, ruler", "766) running shoe", "767) safe", "768) safety pin", "769) saltshaker, salt shaker", "770) sandal", "771) sarong", "772) sax, saxophone", "773) scabbard", "774) scale, weighing machine", "775) school bus", "776) schooner", "777) scoreboard", "778) screen, CRT screen", "779) screw", "780) screwdriver", "781) seat belt, seatbelt", "782) sewing machine", "783) shield, buckler", "784) shoe shop, shoe-shop, shoe store", "785) shoji", "786) shopping basket", "787) shopping cart", "788) shovel", "789) shower cap", "790) shower curtain", "791) ski", "792) ski mask", "793) sleeping bag", "794) slide rule, slipstick", "795) sliding door", "796) slot, one-armed bandit", "797) snorkel", "798) snowmobile", "799) snowplow, snowplough", "800) soap dispenser", "801) soccer ball", "802) sock", "803) solar dish, solar collector, solar furnace", "804) sombrero", "805) soup bowl", "806) space bar", "807) space heater", "808) space shuttle", "809) spatula", "810) speedboat", "811) spider web, spider's web", "812) spindle", "813) sports car, sport car", "814) spotlight, spot", "815) stage", "816) steam locomotive", "817) steel arch bridge", "818) steel drum", "819) stethoscope", "820) stole", "821) stone wall", "822) stopwatch, stop watch", "823) stove", "824) strainer", "825) streetcar, tram, tramcar, trolley, trolley car", "826) stretcher", "827) studio couch, day bed", "828) stupa, tope", "829) submarine, pigboat, sub, U-boat", "830) suit, suit of clothes", "831) sundial", "832) sunglass", "833) sunglasses, dark glasses, shades", "834) sunscreen, sunblock, sun blocker", "835) suspension bridge", "836) swab, swob, mop", "837) sweatshirt", "838) swimming trunks, bathing trunks", "839) swing", "840) switch, electric switch, electrical switch", "841) syringe", "842) table lamp", "843) tank, army tank, armored combat vehicle, armoured combat vehicle", "844) tape player", "845) teapot", "846) teddy, teddy bear", "847) television, television system", "848) tennis ball", "849) thatch, thatched roof", "850) theater curtain, theatre curtain", "851) thimble", "852) thresher, thrasher, threshing machine", "853) throne", "854) tile roof", "855) toaster", "856) tobacco shop, tobacconist shop, tobacconist", "857) toilet seat", "858) torch", "859) totem pole", "860) tow truck, tow car, wrecker", "861) toyshop", "862) tractor", "863) trailer truck, tractor trailer, trucking rig, rig, articulated lorry, semi", "864) tray", "865) trench coat", "866) tricycle, trike, velocipede", "867) trimaran", "868) tripod", "869) triumphal arch", "870) trolleybus, trolley coach, trackless trolley", "871) trombone", "872) tub, vat", "873) turnstile", "874) typewriter keyboard", "875) umbrella", "876) unicycle, monocycle", "877) upright, upright piano", "878) vacuum, vacuum cleaner", "879) vase", "880) vault", "881) velvet", "882) vending machine", "883) vestment", "884) viaduct", "885) violin, fiddle", "886) volleyball", "887) waffle iron", "888) wall clock", "889) wallet, billfold, notecase, pocketbook", "890) wardrobe, closet, press", "891) warplane, military plane", "892) washbasin, handbasin, washbowl, lavabo, wash-hand basin", "893) washer, automatic washer, washing machine", "894) water bottle", "895) water jug", "896) water tower", "897) whiskey jug", "898) whistle", "899) wig", "900) window screen", "901) window shade", "902) Windsor tie", "903) wine bottle", "904) wing", "905) wok", "906) wooden spoon", "907) wool, woolen, woollen", "908) worm fence, snake fence, snake-rail fence, Virginia fence", "909) wreck", "910) yawl", "911) yurt", "912) web site, website, internet site, site", "913) comic book", "914) crossword puzzle, crossword", "915) street sign", "916) traffic light, traffic signal, stoplight", "917) book jacket, dust cover, dust jacket, dust wrapper", "918) menu", "919) plate", "920) guacamole", "921) consomme", "922) hot pot, hotpot", "923) trifle", "924) ice cream, icecream", "925) ice lolly, lolly, lollipop, popsicle", "926) French loaf", "927) bagel, beigel", "928) pretzel", "929) cheeseburger", "930) hotdog, hot dog, red hot", "931) mashed potato", "932) head cabbage", "933) broccoli", "934) cauliflower", "935) zucchini, courgette", "936) spaghetti squash", "937) acorn squash", "938) butternut squash", "939) cucumber, cuke", "940) artichoke, globe artichoke", "941) bell pepper", "942) cardoon", "943) mushroom", "944) Granny Smith", "945) strawberry", "946) orange", "947) lemon", "948) fig", "949) pineapple, ananas", "950) banana", "951) jackfruit, jak, jack", "952) custard apple", "953) pomegranate", "954) hay", "955) carbonara", "956) chocolate sauce, chocolate syrup", "957) dough", "958) meat loaf, meatloaf", "959) pizza, pizza pie", "960) potpie", "961) burrito", "962) red wine", "963) espresso", "964) cup", "965) eggnog", "966) alp", "967) bubble", "968) cliff, drop, drop-off", "969) coral reef", "970) geyser", "971) lakeside, lakeshore", "972) promontory, headland, head, foreland", "973) sandbar, sand bar", "974) seashore, coast, seacoast, sea-coast", "975) valley, vale", "976) volcano", "977) ballplayer, baseball player", "978) groom, bridegroom", "979) scuba diver", "980) rapeseed", "981) daisy", "982) yellow lady's slipper, yellow lady-slipper, Cypripedium calceolus, Cypripedium parviflorum", "983) corn", "984) acorn", "985) hip, rose hip, rosehip", "986) buckeye, horse chestnut, conker", "987) coral fungus", "988) agaric", "989) gyromitra", "990) stinkhorn, carrion fungus", "991) earthstar", "993) bolete", "994) ear, spike, capitulum", "995) toilet tissue, toilet paper, bathroom tissue"]

def interpolate_and_shape(A, B, num_interps):
  """Linearly interpolate between vectors A and B and reshape."""
  interps = interpolate(A, B, num_interps)
  return (interps.transpose(1, 0, *range(2, len(interps.shape)))
                 .reshape(num_samples * num_interps, *interps.shape[2:]))

z_A, z_B = [truncated_z_sample(num_samples, truncation, noise_seed)
            for noise_seed in [noise_seed_A, noise_seed_B]]
y_A, y_B = [one_hot([int(category.split(')')[0])] * num_samples)
            for category in [category_A, category_B]]

z_interp = interpolate_and_shape(z_A, z_B, num_interps)
y_interp = interpolate_and_shape(y_A, y_B, num_interps)

ims = sample(sess, z_interp, y_interp, truncation=truncation)
fig=plt.imshow(imgrid(ims, cols=num_interps))
fig.axes.get_xaxis().set_visible(False)
fig.axes.get_yaxis().set_visible(False)

get_name = lambda name: name.split(")")[1].split(",")[0]

cat_a = get_name(category_A)
cat_b = get_name(category_B)
print("The model is merging a {} with a {}.".format(cat_a, cat_b))
print("Look at the sample in the middle. The model is imagining a '{}-{}'".format(cat_a, cat_b))

# Style Transfer.

**Style transfer** is another example of a generative model exhibiting imagination. In style transfer a generative model applies the style present in a **style image** to the content of a **content** image. Let's have a look at an example below.

In [ ]:
# @title Restart the colab kernel

#@markdown Run this cell and wait for the colab kernel to restart before running the next cell.
#@markdown > There will be a green tick in the top right of the screen when it is time to run the next cell.

import os
os._exit(00)

In [ ]:
#@title Set up the style transfer code
#@markdown This will take about two minutes.
# Import files needed for the style transfer.
%%capture
import functools
import os
from google.colab import files
from PIL import Image as PILImage

from matplotlib import gridspec
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
try:
  tf.executing_eagerly()
except:
  "You need to run the cell above and wait for the colab to restart."
  raise

from tensorflow.keras.utils import get_file
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from IPython.display import Image

from google.colab import widgets
import ipywidgets

# https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/tf2_arbitrary_image_stylization.ipynb#scrollTo=lnAv-F3O9fLV

def crop_center(image):
  """Returns a cropped square image."""
  shape = image.shape
  new_shape = min(shape[1], shape[2])
  offset_y = max(shape[1] - shape[2], 0) // 2
  offset_x = max(shape[2] - shape[1], 0) // 2
  image = tf.image.crop_to_bounding_box(
      image, offset_y, offset_x, new_shape, new_shape)
  return image

@functools.lru_cache(maxsize=None)
def load_image(image_url, image_size=(256, 256), crop_image=True, preserve_aspect_ratio=True):
  """Loads and preprocesses images."""
  # Cache image file locally.
  image_path = tf.keras.utils.get_file(os.path.basename(image_url)[-128:], image_url)
  # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
  img = plt.imread(image_path).astype(np.float32)[np.newaxis, ...]
  if img.max() > 1.0:
    img = img / 255.
  if len(img.shape) == 3:
    img = tf.stack([img, img, img], axis=-1)
  if crop_image:
    img = crop_center(img)
  if image_size is not None:
    img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
  return img

def plot_images(images, titles=None):
  """Plots two or three images each next to one-another."""
  assert len(images) in [2, 3]
  if titles is None:
    title_sequence = [''] * (len(images) * 2 - 1)
  else:
    title_sequence = [titles[0], '', titles[1]]
  image_sequence = [images[0], image_plus, images[1]]
  if len(images) == 3:
    image_sequence.extend([image_arrow, images[2]])
    title_sequence.extend(['', titles[2]])
  
  fig, ax = plt.subplots(1, len(image_sequence), figsize=(20, 60))
  for i, (img, ttl) in enumerate(zip(image_sequence, title_sequence)):
    ax[i].imshow(img[0])
    ax[i].axis('off')
    ax[i].set_title(ttl)


def plot_input_images(images, titles=None):
  """Plots the the content and style image images."""
  # images is a list [content image, style image]
  # titles is a list [content title, style title]
  if titles is None:
    title_sequence = [''] * (len(images) * 2 - 1)
  else:
    title_sequence = [titles[0], '', titles[1], '']
  image_sequence = [images[0], image_plus, images[1], image_arrow]
  
  fig, ax = plt.subplots(1, len(image_sequence), figsize=(20, 60))
  for i, (img, ttl) in enumerate(zip(image_sequence, title_sequence)):
    ax[i].imshow(img[0])
    ax[i].axis('off')
    ax[i].set_title(ttl)

#@title Preloading images and model
plus_sign_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/c/ce/Plus_font_awesome.svg/200px-Plus_font_awesome.svg.png'
arrow_sign_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/7/71/Arrow_east.svg/200px-Arrow_east.svg.png'
image_plus_temp = load_image(plus_sign_url, None, crop_image=False)
image_arrow_temp = load_image(arrow_sign_url, None, crop_image=False)

# scaling the images down
image_plus = np.zeros([1, 400, 400, 4])
image_plus[:, 100:300, 100:300, :] = image_plus_temp

image_arrow = np.zeros([1, 162, 400, 4])
image_arrow[:, 40:121, 100:300, :] = image_arrow_temp


# load pretrained model
hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

def _imshow(im, title=None):
  fig=plt.imshow(im)
  fig.axes.get_xaxis().set_visible(False)
  fig.axes.get_yaxis().set_visible(False)
  if title is not None:
    plt.title(title)
  plt.show()
  return fig

def download_image(inputs=None, filename='stylized_image.jpeg', image_array='None'):
  assert image_array is not None
  img = (255 * image_array.numpy()).astype(np.uint8)
  im = PILImage.fromarray(img)
  im.save(filename)
  files.download(filename) 


In [ ]:
# @title Load your content and style image. { run: "auto" }
#@markdown Select a `content_image` and a `style_image` from the drop down menu, then run the cell below. This will show your `Content Image` and your `Style Image`. An `Apply style transfer` button will appear, click this and see the new `Stylized image` appear!

fixed_choices = {
    "Van Gogh - The Starry Night": "https://www.vangoghgallery.com/img/starry_night_full.jpg",
    "Van Gogh - Self-portrait with grey felt hat": "https://upload.wikimedia.org/wikipedia/commons/thumb/a/aa/Vincent_van_Gogh_-_Self-portrait_with_grey_felt_hat_-_Google_Art_Project.jpg/988px-Vincent_van_Gogh_-_Self-portrait_with_grey_felt_hat_-_Google_Art_Project.jpg",
    "Frida Kahlo - Viva la Vida, Watermelons": "https://uploads1.wikiart.org/images/magdalena-carmen-frieda-kahlo-y-calder%C3%B3n-de-rivera/viva-la-vida-watermelons.jpg!Large.jpg",
    "Joan Miro - The Smile of the Flamboyant Wings": "https://uploads5.wikiart.org/images/joan-miro/the-smile-of-the-flamboyant-wings.jpg",
    "Jean-Michel Basquiat - Skull": "https://uploads5.wikiart.org/images/jean-michel-basquiat/head.jpg!Large.jpg",
    "Piet Mondrian - Composition No IV": "https://upload.wikimedia.org/wikipedia/commons/thumb/6/66/Composition_No_IV%2C_by_Piet_Mondriaan.jpg/836px-Composition_No_IV%2C_by_Piet_Mondriaan.jpg",
    "Piet Mondrian - Blossoming apple tree": "https://upload.wikimedia.org/wikipedia/commons/thumb/0/03/Blossoming_apple_tree%2C_by_Piet_Mondriaan.jpg/1599px-Blossoming_apple_tree%2C_by_Piet_Mondriaan.jpg",
    "Andy Warhol - Liz Taylor": "https://uploads2.wikiart.org/images/andy-warhol/liz-taylor.jpg!Large.jpg",
    "Georgia O'Keeffe - Blue and Green Music": "https://upload.wikimedia.org/wikipedia/commons/2/26/Blue-green.jpg",
    "Georgia O'Keeffe - Oriental Poppies": "https://upload.wikimedia.org/wikipedia/en/c/ca/Red_Canna_%281924%29_by_Georgia_O%27Keeffe.jpg",
    "Wassily Kandinsky, 1923 - On White II": "https://upload.wikimedia.org/wikipedia/commons/c/c4/Vassily_Kandinsky%2C_1923_-_On_White_II.jpg",
    "Wassily Kandinsky - Several Circles" : "https://i0.wp.com/www.guggenheim.org/wp-content/uploads/1926/01/41.283_ph_web-1.jpg",
    "Patrick Hoesly - Matrix iPhone Background": "https://live.staticflickr.com/5052/5508849065_c312d5cd28_w.jpg",
    }

fixed_content_images = {
    "House": "https://upload.wikimedia.org/wikipedia/commons/thumb/6/67/Strada_Romulus_nr._77_%28B-II-m-B-19616%29.JPG/354px-Strada_Romulus_nr._77_%28B-II-m-B-19616%29.JPG",
    "Corgi Dog": "https://upload.wikimedia.org/wikipedia/commons/1/12/ThreeTimeAKCGoldWinnerPembrookeWelshCorgi.jpg",
    "Cat": "https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Felis_catus-cat_on_snow.jpg/220px-Felis_catus-cat_on_snow.jpg",
    "Beach": "https://upload.wikimedia.org/wikipedia/commons/6/62/Makapu%CA%BBu_Beach.JPG",
    "Tree": "https://upload.wikimedia.org/wikipedia/commons/e/eb/Ash_Tree_-_geograph.org.uk_-_590710.jpg"
}

content_image = "Tree" #@param ["House", "Corgi Dog", "Cat", "Beach", "Tree"]

style_image = "Piet Mondrian - Blossoming apple tree" #@param ["Van Gogh - The Starry Night", "Van Gogh - Self-portrait with grey felt hat", "Frida Kahlo - Viva la Vida, Watermelons", "Joan Miro - The Smile of the Flamboyant Wings", "Jean-Michel Basquiat - Skull", "Piet Mondrian - Composition No IV", "Piet Mondrian - Blossoming apple tree", "Andy Warhol - Liz Taylor", "Georgia O'Keeffe - Blue and Green Music", "Georgia O'Keeffe - Oriental Poppies", "Wassily Kandinsky, 1923 - On White II", "Wassily Kandinsky - Several Circles", "Patrick Hoesly - Matrix iPhone Background"] {allow-input: true}

blur_style_image = True

if style_image in fixed_choices:
  style_image = fixed_choices[style_image]

if content_image in fixed_content_images:
  content_image = fixed_content_images[content_image]

content_image = load_image(content_image, None)
style_image = load_image(style_image, (256, 256))
# blurring the style image for a prettier transfer

if blur_style_image:
  style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')

outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
stylized_image = outputs[0]

def show_style_transfer(inputs=None):
  print("Applying style transfer.")
  with grid.output_to(0, 1):
    style_button.disabled = False
    _imshow(stylized_image[0], "Stylized image.")

grid = widgets.Grid(1, 2)
with grid.output_to(0, 0):
  plot_input_images([content_image, style_image], ['Content image', 'Style image'])



style_button = ipywidgets.Button(description="Apply style transfer")
display(style_button)
style_button.on_click(show_style_transfer)

download_stylised_image = functools.partial(download_image, filename='stylized_image.jpeg', image_array=stylized_image[0])
download_button = ipywidgets.Button(description="Download stylised image")
display(download_button)
download_button.on_click(download_stylised_image)


# See how Artist evolves and improves over time.

The process of the Artist and Detective competing against one-another, with the Artist producing more realistic images and the Detective getting better at distinguishing the real images from the fake ones, may be more formally referred to as **training**. Training the Artist to synthesise natural images can take a long time so we have trained one for you.

In this section you will see what kinds of images the Artist learns to generate while it is training and how it improves with more training steps.

In [ ]:
#@title Imports
%%capture
'''
We're reading the pickle files, each pickle file has 18MB and contains:
  dict_keys(['gen_imgs', 'disc_fake', 'IS_mean', 'IS_std', 'FID']) for a given step.
'''
import glob
import dill
import IPython
import requests
from io import BytesIO
import urllib
from matplotlib import pyplot as plt
from google.colab import widgets
from matplotlib import pylab
import numpy as np
from matplotlib import rc
rc('animation', html='jshtml')
import matplotlib.animation as animation

In [ ]:
#@title Read the saved data.
BASE_PATH = 'https://storage.googleapis.com/dm-educational/assets/generative/bigGAN/1'

FOLDERS = ['100040', '100829', '101618', '102407', '103196', '103985', '104774',
 '105563', '107141', '10727', '107929', '108718', '109504', '110293', '111082',
 '111871', '112660', '114238', '115027', '115816', '116604', '117393', '118182',
 '118971', '120548', '121337', '122126', '122915', '12305', '123704', '124493',
 '125282', '126071', '1261', '126860', '128438', '129227', '130016', '130805',
 '13094', '131594', '132383', '133172', '134749', '135538', '136325', '137114',
 '137903', '138691', '13883']

def update_progress_bar(int_percentage):
  IPython.display.clear_output(wait=True)
  bar = '#' * int_percentage + '.' * (100 - int_percentage)
  print('Reading data:', bar, ' %d%%' % int_percentage)

def get_data():
  folders = glob.glob('bigGAN/1/*')
  res = {}
  for i, folder_id in enumerate(FOLDERS):
    try:
      step = int(folder_id)
      path = f'{BASE_PATH}/{folder_id}/gen_ed.pkl'
      print(path)
      res[step] = dill.load(urllib.request.urlopen(path))
    except (ValueError, FileNotFoundError):
      # It could be that some files were not loaded but it shouldn't be a problem.
      pass
    int_percentage = int(i / len(FOLDERS) * 100)
    update_progress_bar(int_percentage)
  return res

res = get_data()
update_progress_bar(100)
sorted_keys = sorted(res.keys())

scores = []
for key in sorted_keys:
  scores.append(res[key]['IS_mean'])

# Check that we at least read some files.
assert len(sorted_keys) >= 10, 'There was some error when loading the requested files.'

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [ ]:
# @title Explore how the Artist evolves over time. { run: "auto" }


#@markdown Slide the "training_percentage" from left to right to see the Artist get better over time.

#@markdown (1) To look at the images generated at the start of training move the slider to the far left. See that the images are not very realistic.

#@markdown (2) To look at the images generated towards the end of training move the slider to the far right. Notice that the images generated are very realistic.

#@markdown With each image we also report the "Detective Prediction". If the detective thinks that the image is real this value will be close to one, while if the detective thinks that the image is fake it will be close to zero.

training_percentage = 2 #@param {type:"slider", min:0, max:100, step:1}
training = min(int(training_percentage / 100. * len(sorted_keys)), len(sorted_keys) - 1)
num_samples = 9

data = []
key = sorted_keys[training]
indices = [(0, 2), (0, 1), (7, 0), (4, 2),
           (2, 2), (2, 6), (3, 7), (4, 3),
           (6, 7), (0, 7), (6, 3), (7, 1),
           (0, 0), (4, 6), (5, 1), (5, 2)]
for n_sample in range(num_samples):
  img = np.clip(res[key]['gen_imgs'], 0, 1)
  i, j = indices[n_sample]
  plt.show()
  data.append(
      {
          'img': np.transpose(img[i][j], (1, 2, 0)),
          'pred': sigmoid(res[key]['disc_fake'][i][j]),
          'is_score': scores[training],
          'step': int(key),
      }
  )


# Plot performance.
plt.figure(figsize=(10, 6))
plt.plot(sorted_keys, scores)
plt.xlabel('Training steps')
plt.ylabel('Performance (Inception Score, higher is better)')
plt.plot([d['step'] for d in data], [d['is_score'] for d in data], 'rx')
plt.show()
print('\n' * 2)


# Plot grid with images.
rows, cols = int(np.sqrt(num_samples)), int(np.sqrt(num_samples))
grid = widgets.Grid(rows, cols)
for i, (row, col) in enumerate(grid):
  pylab.figure(figsize=(4, 4))
  pylab.imshow(data[i]['img'])
  pylab.yticks([])
  pylab.xticks([])
  title = ' ' * 5 + 'Step: %d' % data[i]['step'] + '\n'
  title += 'Detective Prediction: %.2f' % data[i]['pred']
  pylab.title(title)

In [ ]:
# @title Watch how the Artist improves over time.
#@markdown Press play to run the animation. You can run this cell again to see a different image over time.

i, j = np.random.randint(0, 8, size=(2,))

data = []
for index, key in enumerate(sorted_keys):
  if index % (len(sorted_keys) // 10) == 0:
    img = res[key]['gen_imgs']
    plt.show()
    data.append(
        {
            'img': np.transpose(img[i][j], (1, 2, 0)),
            'pred': sigmoid(res[key]['disc_fake'][i][j]),
            'is_score':  scores[index],
            'step': int(key),
        }
    )

fig = plt.figure()
images = []

for i in range(len(data)):
    im = plt.imshow(np.clip(data[i]['img'], 0, 1), animated=True)
    plt.xticks([])
    plt.yticks([])
    images.append([im])

ani = animation.ArtistAnimation(fig, images, interval=1000, blit=True,
                                repeat_delay=1000)
plt.close()
print('Run this cell again to generate new images!')
ani

In the next section, you will have a chance to try training your own Artist and Detective.

# Try it for yourself.

In this section we will show you how to set up the Artist and Detective.

If you were to dig into the code in the cells below you would notice that rather than having an Artist and a Detective we have a **Generator** (aka the Artist) and a **Discriminator** (aka the detective). These are the more "technical" terms for these components.



In [ ]:
# @title Run this cell and wait for the colab to re-start. 
# Since we're using TensorFlow 2 for this part of the colab we need to restart the kernel and install / import dependencies again.
# @markdown You will need to wait for the colab to restart before running the next cell. There will be a green tick in the top right of the screen when it is time to run the next cell.

import os
os._exit(00)

We are going to train a generative model to generate an image similar to one that you draw.

Before we start training the generative model, we first need some data, in this case a single image, to train our generative model on. This image will serve as the **real** example that the:

1.   Generator (Artist) is trying to replicate your artwork.
2.   Discriminator (Detective) is trying to distinguish your drawing from the Generator's output.

Let's capture your drawing for training. Run the cell below and draw something on the canvas that appears? You could draw anything, even a smiley face. 

Once you have drawn the image press the save button.

In [ ]:
#@title Create a canvas to draw an image on.
#@title Run this cell to import things needed to train your own generative model.
import tensorflow as tf
assert tf.executing_eagerly()

from io import BytesIO
from io import StringIO
from PIL import Image
import base64

import IPython
from IPython.display import display
from google.colab import output
import matplotlib.image as mpimg
from matplotlib import pylab
from skimage.transform import resize
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import PIL
import time
from tensorflow.keras import layers

import uuid
from google.colab import widgets

display(IPython.display.HTML('''
    <div id='maindiv'>
    <canvas id='canvas' width="300" height="300"
    style="border:3px solid #616161;"></canvas>
    </div>
    <button id='clear'>Clear Canvas</button>
    <img id='img'></img>

<script>
TRACK_MOUSE = false

function clearCanvas() {
  var canvas = document.getElementById('canvas');
  var ctxt2d = canvas.getContext("2d");
  ctxt2d.clearRect(0, 0, 300, 300);
}

function mouseDownEvent(mouseEvent) {
  TRACK_MOUSE = true;
}

function mouseUpEvent(mouseEvent) {
  TRACK_MOUSE = false;
}

IMG_DATA = null

function mouseMoveEvent(mouseEvent) {
  if (TRACK_MOUSE) {
    var square = document.createElement("div");
    square.classList.add('square');
    var canvas = document.getElementById('canvas');
    var ctxt2d = canvas.getContext("2d");
    ctxt2d.beginPath();

    var pointerX = mouseEvent.clientX - canvas.offsetLeft;
    var pointerY = mouseEvent.clientY - canvas.offsetTop
    ctxt2d.arc(pointerX, pointerY, 12, 0, 2 * Math.PI);
    ctxt2d.fill();

    console.log(mouseEvent.clientX - canvas.offsetLeft);
    console.log(mouseEvent.clientY - canvas.offsetTop);

  }
}

function saveImage(e) {
  var canvas = document.getElementById('canvas');
  var ctx = canvas.getContext("2d");
  console.log(canvas.offsetLeft);
  var img = ctx.getImageData(0, 0, 300, 300);
  IMG_DATA = canvas.toDataURL();
  return canvas.toDataURL();
}
document.getElementById('clear').addEventListener("click", clearCanvas);

document.getElementById('canvas').addEventListener("mousemove", mouseMoveEvent);
document.getElementById('canvas').addEventListener("mousedown", mouseDownEvent);
document.getElementById('canvas').addEventListener("mouseup", mouseUpEvent);
</script>
'''))

class InvokeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback

  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """<button id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

image = None
def do_something():
  global image
  data_url = output.eval_js('saveImage()')
  header, encoded = data_url.split(",", 1)
  data = base64.b64decode(encoded)
  with open("image.png", "wb") as f:
    f.write(data)
    print('Image saved successfully!')

InvokeButton('Save Image', do_something)

In the next cell we are going to look at the image we have captured and resize the image to make it a little bit smaller. This helps the model to learn faster.

In [ ]:
#@title Show the input image and process the image to make it smaller.
def load_image():
  img = mpimg.imread('image.png')
  img = img[:,:,3] # remove unused channels.

  fig, axes = plt.subplots(1, 2)
  axes[0].set_title('Original Image')
  axes[0].imshow(img, cmap='Greys')
  axes[0].get_xaxis().set_visible(False)
  axes[0].axes.get_yaxis().set_visible(False)
  # plt.show()

  input_img = resize(img, (28, 28)) #/ 255.
  axes[1].set_title('Preproccesed image')
  axes[1].imshow(input_img, cmap='Greys')
  axes[1].get_xaxis().set_visible(False)
  axes[1].get_yaxis().set_visible(False)

  return input_img

img = load_image()

Now that we have the data let's create our Artist and Detective.



In [ ]:
#@title Create the Artist and the Detective.
# Based on https://twitter.com/fchollet/status/1250622989541838848

#@title Create the Artist and the Detective.
# Based on https://twitter.com/fchollet/status/1250622989541838848

loss_fn = tf.losses.BinaryCrossentropy(from_logits=True)


class GeneratorModel(tf.keras.Model):
  def __init__(self, dense_units=[32, 128], final_layer=784, latent_dim=100):
    super().__init__()
    self.latent_dim = latent_dim
    self.dense_layers = [tf.keras.layers.Dense(u, activation=tf.nn.relu) for u in dense_units]
    self.final_layer = tf.keras.layers.Dense(final_layer, activation=tf.nn.sigmoid)

  def call(self, x):
    for dense_layer in self.dense_layers:
      x = dense_layer(x)
    return self.final_layer(x)

class DiscriminatorModel(tf.keras.Model):
  def __init__(self, dense_units=[784, 256, 128, 16], final_layer=1):
    super().__init__()
    self.dense_layers = [tf.keras.layers.Dense(u, activation=tf.nn.relu) for u in dense_units]
    self.final_layer = tf.keras.layers.Dense(final_layer, activation=None)

  def call(self, x):
    for dense_layer in self.dense_layers:
      x = dense_layer(x)
    return self.final_layer(x)

class GAN(tf.keras.Model):

  def __init__(self, generator_net, discriminator_net):
    super().__init__()
    self.generator_net = generator_net
    self.discriminator_net = discriminator_net

    self.gen_train_loss = []
    self.disc_train_loss = []

  def compile(self, generator_opt, discriminator_opt, loss_fn):
    super().compile()
    self.generator_opt = generator_opt
    self.discriminator_opt = discriminator_opt
    self.loss_fn = loss_fn
    self.generator_net.compile()
    self.discriminator_net.compile()

  def update_net(self, data, labels, net, optimizer):
    with tf.GradientTape() as tape:
      preds = net(data)
      loss = self.loss_fn(labels, preds)
    grads = tape.gradient(loss, net.trainable_weights)
    optimizer.apply_gradients(zip(grads, net.trainable_weights))
    return loss

  def train_step(self, real_data):
    batch_size = tf.shape(real_data)[0]
    
    # Generate fake data.
    gen_data = self.sample(batch_size=batch_size)

    # Train discriminator.
    data = tf.concat([real_data, gen_data], axis=0)
    labels = tf.concat([tf.ones((batch_size,1)), tf.zeros((batch_size,1))], axis=0)
    discriminator_loss = self.update_net(data, labels,
                                         self.discriminator_net,
                                         self.discriminator_opt)
    self.disc_train_loss.append(discriminator_loss)
    
    # Train generator.
    all_real_labels = tf.ones((batch_size,1))
    with tf.GradientTape() as tape:
      preds = self.discriminator_net(self.sample(batch_size=batch_size))
      generator_loss = self.loss_fn(all_real_labels, preds)
    grads = tape.gradient(generator_loss, self.generator_net.trainable_weights)
    self.generator_opt.apply_gradients(zip(grads, self.generator_net.trainable_weights))
    self.gen_train_loss.append(generator_loss)

    return {'d_loss': discriminator_loss, 'g_loss': generator_loss}

  def sample(self, batch_size=1, z=None):
    if z is None:
      z = tf.random.normal(shape=(batch_size, self.generator_net.latent_dim))
    return self.generator_net(z)

Run the cell below and watch the Artist get better over time. At the start of the game the Artist outputs noise after a few steps the model gets much better and the image resembles the one you drew above.

You can also change the **`number_of_training_steps`** to see how this affects the Artist skills. If the artist trains for very few steps (like 10) you'll see that they are not able to replicate your art, but you should see that when you increase this number it should get better.

In [ ]:
# @title Look at the Artist getting better over time.

number_of_training_steps = 94 #@param {type:"slider", min:10, max:200, step:1}

# Instantiate the artist (generator)
generator_net = GeneratorModel()
# Instantiate the detective (discriminator)
discriminator_net = DiscriminatorModel()

gan_net = GAN(generator_net, discriminator_net)
gan_net.compile(tf.keras.optimizers.Adam(), tf.keras.optimizers.Adam(),
                tf.keras.losses.BinaryCrossentropy(from_logits=True))

# Run the model and show the images.
num_runs = 10
grid = widgets.Grid(2, 5)
for i, (row, col) in enumerate(grid):
  if i > 0:
    gan_net.fit(np.reshape(img, (1, -1)), epochs=number_of_training_steps // num_runs, verbose=False)
  pylab.figure(figsize=(4, 4))
  pylab.imshow(gan_net.sample().numpy().reshape((28, 28)), cmap='Greys')
  pylab.yticks([])
  pylab.xticks([])
  pylab.title('Time: %d' % i)

This generative model that you just trained is only able to generate images that are very similar to your drawing because that's all the data that it has ever seen!

Typically generative models are trained with millions of images and this allows them to exploit patterns in the images to generate many different images, even ones not seen in the training data.